# Initialization

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
import surprise

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

In [5]:
def mdir(obj):
    print(*[i for i in dir(obj) if not i.startswith("_")], sep="\n")

# Загрузка данных

In [6]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

В качестве точки разбиения используем `2017-08-01`, то есть отнесем в тестовую часть три последних месяца.

In [7]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = np.intersect1d(users_train, users_test)

print(len(users_train), len(users_test), len(common_users)) 

428220 123223 120858


---

# === Знакомство: "холодный" старт

## "Холодные" пользователи

«Холодные» пользователи — те, которые есть в `test`, но отсутствуют в `train`. Это соответствует хронологическому порядку, в котором и работает рекомендательная система.

In [8]:
cold_users = np.setdiff1d(users_test, users_train)

print(len(cold_users)) 

2365


## Топ-100 наиболее популярных книг

Теперь можно построить сами рекомендации. Для этого найдем топ-100 наиболее популярных книг (учитывая и их среднюю оценку) за последние несколько лет. Например, с 2015 года и со средней оценкой не меньше 4. Их мы будем рекомендовать «холодным» пользователям.

In [9]:
top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()

In [10]:
item_popularity.head()

,item_id,users,avg_rating
0,1,8728,4.796059
1,2,9528,4.685873
2,3,15139,4.706057
3,5,11890,4.770143
4,6,10679,4.764772


In [11]:
item_popularity["popularity_weighted"] = item_popularity["users"] * item_popularity["avg_rating"]

In [12]:
# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values("popularity_weighted", ascending=False)

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity[item_popularity["avg_rating"] > 4].head(100)

In [13]:
top_k_pop_items

,item_id,users,avg_rating,popularity_weighted
32387,18007564,20207,4.321275,87320.0
32623,18143977,19462,4.290669,83505.0
30695,16096824,16770,4.301014,72128.0
2,3,15139,4.706057,71245.0
3718,38447,14611,4.232770,61845.0
...,...,...,...,...
19596,2767052,4361,4.413437,19247.0
32835,18293427,4674,4.092640,19129.0
378,3636,4667,4.098564,19128.0
33611,18966819,4361,4.374914,19079.0


Добавив информацию о книгах, можно просмотреть, какие попали в топ.

In [14]:
# добавляем информацию о книгах

top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_weighted", "genre_and_votes"]]) 


,item_id,author,title,publication_year,users,avg_rating,popularity_weighted,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,87320.0,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,83505.0,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,72128.0,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
3,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,71245.0,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
4,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,61845.0,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
5,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,60421.0,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14348,4.179189,59963.0,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12177,4.730640,57605.0,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,18692431,"Nicola Yoon, David Yoon","Everything, Everything",2015,14121,4.071454,57493.0,"{'Young Adult': 5175, 'Romance': 3234, 'Contem..."
9,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11890,4.770143,56717.0,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."


## Рекомендации

Топ-рекомендации назовём рекомендациями по умолчанию (англ. default recommendations).

Уже можно оценить их качество — для этого их следует соотнести с событиями из теста, как будто бы вы показываете эти рекомендации «холодным» пользователям, зная их предпочтения.

In [15]:
# соотнесем события из теста с топ-рекомендациями

cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items[["item_id", "avg_rating"]], on="item_id", how="left")

In [16]:
cold_users_events_with_recs.shape

(9672, 10)

In [17]:
cold_users_events_with_recs.head()

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month,user_id_old,avg_rating
0,1000153,29236299,2017-08-16,2017-08-20,True,5,False,2017-08-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN
1,1000153,26114463,2017-08-20,2017-09-09,True,5,False,2017-08-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN
2,1000153,13262783,2017-08-12,2017-08-14,True,5,False,2017-08-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN
3,1000153,29923707,2017-10-08,2017-10-13,True,4,False,2017-10-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN
4,1000153,18806259,2017-08-08,2017-08-12,True,3,False,2017-08-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN


In [18]:
# найдем клиентов с покрытием по рекомендациям

cold_user_recs = cold_users_events_with_recs.loc[
    cold_users_events_with_recs["avg_rating"].notna(), ["user_id", "item_id", "rating", "avg_rating"]
]

In [19]:
cold_user_recs.shape

(1912, 4)

In [20]:
cold_user_recs.head()

,user_id,item_id,rating,avg_rating
13,1000504,1885,5,4.316316
17,1000712,13496,5,4.440779
23,1001508,1885,5,4.316316
25,1002222,18966819,5,4.374914
27,1002222,15839976,4,4.150180


In [21]:
# Для какой доли событий «холодных» пользователей в events_test рекомендации в top_k_pop_items совпали по книгам

cold_users_events_with_recs["avg_rating"].notna().mean()

0.19768403639371382

## Метрики

Расчитаем метрики `rmse` и `mae` для полученных рекомендаций.

In [22]:
cold_user_recs.head()

,user_id,item_id,rating,avg_rating
13,1000504,1885,5,4.316316
17,1000712,13496,5,4.440779
23,1001508,1885,5,4.316316
25,1002222,18966819,5,4.374914
27,1002222,15839976,4,4.150180


In [23]:
rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])

print("RMSE =", round(rmse, 3))
print(" MAE =", round(mae, 3))

RMSE = 0.776
 MAE = 0.625


### Покрытие холодных пользователей рекомендациями

Полученная метрика — значение, на которое оценка рекомендации в среднем отклоняется от истинной

Это отклонение выглядит небольшим, но в данном случае стоит оценить покрытие как по пользователям, так и по объектам, ведь рекомендации, скорее всего, имеют мало пересечений с оценками пользователей. 

In [24]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (x.notna()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio['hits'] == 0).mean():.2f}")
print(f"Доля пользователей с релевантными рекомендациями: {(cold_users_hit_ratio['hits'] > 0).mean():.2f}")
print(f"\nСреднее покрытие пользователей: {cold_users_hit_ratio.loc[cold_users_hit_ratio['hits'] != 0, 'hits'].mean():.2f}")

Доля пользователей без релевантных рекомендаций: 0.59
Доля пользователей с релевантными рекомендациями: 0.41

Среднее покрытие пользователей: 0.44


«Холодных» пользователей без каких-либо релевантных рекомендаций — 59%, то есть пересечение между оценёнными книгами и рекомендациями есть только у 41%, по ним же и получено значение MAE-метрики. При этом среднее покрытие — 44%. Это значит, что большая часть «холодных» пользователей не получила никаких релевантных рекомендаций, а оставшаяся часть имеет пересечения только по 44% книг. 

---

# === Знакомство: первые персональные рекомендации

Степень разреженности матрицы оценок

In [25]:
# Степень разреженности матрицы можно оценить как отношение незаполненных ячеек к общему их количеству:

sparsity = 1 - events[["user_id", "item_id"]].drop_duplicates().shape[0] / (events["user_id"].nunique() * events["item_id"].nunique())
sparsity


0.9993451160571009

## Реализация SVD-алгоритма

Воспользуемся готовой реализацией SVD-алгоритма из библиотеки `surprise`. В качестве разбиения данных на `train` и `test` возьмём разбиение из предыдущего урока: `events_train`, `events_test`.

In [26]:
mdir(surprise)

AlgoBase
BaselineOnly
CoClustering
Dataset
KNNBaseline
KNNBasic
KNNWithMeans
KNNWithZScore
NMF
NormalPredictor
Prediction
PredictionImpossible
Reader
SVD
SVDpp
SlopeOne
Trainset
accuracy
builtin_datasets
dataset
dump
get_dataset_dir
get_distribution
model_selection
prediction_algorithms
reader
similarities
trainset
utils


In [27]:
# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise

reader = surprise.Reader(rating_scale=(1, 5))
surprise_train_set = surprise.Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)

In [28]:
print(surprise_train_set)
print()
mdir(surprise_train_set)


build_full_trainset
construct_testset
construct_trainset
df
has_been_split
load_builtin
load_from_df
load_from_file
load_from_folds
raw_ratings
read_ratings
reader


In [29]:
surprise_train_set = surprise_train_set.build_full_trainset()

In [30]:
print(surprise_train_set)
print()
mdir(surprise_train_set)


all_items
all_ratings
all_users
build_anti_testset
build_testset
global_mean
ir
knows_item
knows_user
n_items
n_ratings
n_users
rating_scale
to_inner_iid
to_inner_uid
to_raw_iid
to_raw_uid
ur


In [31]:
# инициализируем модель

svd_model = surprise.SVD(n_factors=100, random_state=0)

In [32]:
# обучаем модель
# 2 min

svd_model.fit(surprise_train_set)

Используя обученную модель, получим рекомендации для тестовой выборки:

In [33]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set) 

In [34]:
type(svd_predictions)

list

In [35]:
len(svd_predictions)

424962

In [36]:
svd_predictions[:3]

[Prediction(uid=1000003, iid=25893709, r_ui=4, est=3.146647541664683, details={'was_impossible': False}),
 Prediction(uid=1000005, iid=34076952, r_ui=5, est=4.0248936128868875, details={'was_impossible': False}),
 Prediction(uid=1000006, iid=18774964, r_ui=4, est=4.703536489983392, details={'was_impossible': False})]

## Оценка рекомендаций

Полученные рекомендации можно оценить, используя встроенный модуль `accuracy` из библиотеки `surprise`:

In [37]:
rmse = surprise.accuracy.rmse(svd_predictions)
mae = surprise.accuracy.mae(svd_predictions)

print()
print(rmse, mae) 

RMSE: 0.8263
MAE:  0.6460

0.826346375350908 0.6460143973270805


### Проверка метрик на адекватность

Для генерации случайных рекомендаций библиотека surprise предлагает класс NormalPredictor, который выдаёт случайные рейтинги из нормального распределения.

In [38]:
# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = surprise.NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set) 

In [39]:
len(random_predictions)

424962

In [40]:
random_predictions[:3]

[Prediction(uid=1000003, iid=25893709, r_ui=4, est=5, details={'was_impossible': False}),
 Prediction(uid=1000005, iid=34076952, r_ui=5, est=4.3272637894387875, details={'was_impossible': False}),
 Prediction(uid=1000006, iid=18774964, r_ui=4, est=4.8785393008980575, details={'was_impossible': False})]

In [41]:
rmse_dummy = surprise.accuracy.rmse(random_predictions)
mae_dummy = surprise.accuracy.mae(random_predictions)

print()
print(rmse_dummy, mae_dummy) 

RMSE: 1.2590
MAE:  0.9982

1.2590325375790072 0.9982060614713011


In [42]:
# На сколько процентов MAE для случайных рекомендаций от NormalPredictor выше значения MAE от SVD?

print(f"{mae_dummy / mae - 1:.2%}")

54.52%


## Рекомендации (предсказания)

In [43]:
mdir(svd_model)

bi
biased
bsl_options
bu
compute_baselines
compute_similarities
default_prediction
estimate
fit
get_neighbors
init_mean
init_std_dev
lr_bi
lr_bu
lr_pu
lr_qi
n_epochs
n_factors
predict
pu
qi
random_state
reg_bi
reg_bu
reg_pu
reg_qi
sgd
sim_options
test
trainset
verbose


In [44]:
USER_ID = 1296647

In [45]:
items_to_predict = events['item_id'].unique()

In [46]:
items_to_predict.shape

(41673,)

In [47]:
scores = [svd_model.predict(USER_ID, item_id).est for item_id in items_to_predict]

In [48]:
predictions = pd.DataFrame({
    "item_id": items_to_predict,
    "scores": scores
}).sort_values('scores', ascending=False).reset_index(drop=True)

In [49]:
predictions

,item_id,scores
0,8471387,5.000000
1,24812,5.000000
2,481749,5.000000
3,30688013,4.996900
4,1108124,4.979711
...,...,...
41668,5557442,2.360372
41669,783291,2.254230
41670,6017893,2.215569
41671,16075935,2.208172


In [50]:
def get_recommendations_svd(user_id, events, model, include_seen=True, n=5):

    """ возвращает n рекомендаций для user_id """
    
    # получим список идентификаторов всех книг
    all_items = events['item_id'].unique()
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = all_items
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = events.loc[events["user_id"]==user_id, 'item_id'].unique()
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = np.setdiff1d(all_items, seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    scores = [model.predict(USER_ID, item_id).est for item_id in items_to_predict]
    
    predictions = pd.DataFrame({
        "item_id": items_to_predict,
        "scores": scores
    }).sort_values('scores', ascending=False).reset_index(drop=True).head(n)
    
    return predictions

In [51]:
get_recommendations_svd(USER_ID, events_train, svd_model, False, n=10)

,item_id,scores
0,8471387,5.000000
1,24812,5.000000
2,481749,5.000000
3,30688013,4.996900
4,1108124,4.979711
5,6898978,4.969303
6,24019187,4.964964
7,191139,4.947385
8,11221285,4.946124
9,862041,4.942287


## Дополнительная проверка качества рекомендаций

Некоторые проблемы рекомендаций отследить с помощью метрик сложно.

Например, с помощью метрик сложно выявить рекомендации с низким разнообразием — когда пользователю, который прочитал книгу одного автора, рекомендуются книги преимущественно того же автора.

Такие проблемы проще отследить «глазами». 

In [52]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

user_id: 1109853


In [53]:
print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
0,Patrick Rothfuss,"The Wise Man's Fear (The Kingkiller Chronicle,...",2015-06-17,2015-08-31,5,"{'Fantasy': 16491, 'Fiction': 2222, 'Fantasy-E..."
1,Andy Weir,The Martian,2014-12-07,2014-12-11,4,"{'Science Fiction': 11966, 'Fiction': 8430}"
2,Brandon Sanderson,"The Way of Kings (The Stormlight Archive, #1)",2015-08-31,2015-10-30,5,"{'Fantasy': 14291, 'Fiction': 1623, 'Fantasy-E..."
3,Brandon Sanderson,"Words of Radiance (The Stormlight Archive, #2)",2015-10-30,2016-03-17,5,"{'Fantasy': 8542, 'Fiction': 872, 'Fantasy-Epi..."
4,Ken Follett,"Fall of Giants (The Century Trilogy, #1)",2016-05-12,2016-08-30,4,"{'Historical-Historical Fiction': 4665, 'Ficti..."


In [55]:
print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, events_train, svd_model, n=10)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 

Рекомендации


,item_id,scores,author,title,genre_and_votes
0,8471387,5.000000,أمل دنقل,لا تصالح,{'Poetry': 110}
1,24812,5.000000,Bill Watterson,The Complete Calvin and Hobbes,"{'Sequential Art-Comics': 867, 'Humor': 378, '..."
2,481749,5.000000,James E. Talmage,Jesus the Christ,"{'Religion': 451, 'Christianity-Lds': 256, 'No..."
3,30688013,4.996900,Robin Hobb,"Assassin's Fate (The Fitz and the Fool, #3)","{'Fantasy': 1657, 'Fiction': 172, 'Fantasy-Epi..."
4,1108124,4.979711,Kalki,பொன்னியின் செல்வன் [Ponniyin Selvan],{'Historical-Historical Fiction': 38}
5,6898978,4.969303,"Leo Tolstoy, Rubens Figueiredo",Anna Kariênina,"{'Classics': 18053, 'Fiction': 6990, 'Cultural..."
6,24019187,4.964964,Brandon Stanton,Humans of New York: Stories,"{'Nonfiction': 662, 'Art-Photography': 190, 'A..."
7,191139,4.947385,Dr. Seuss,"Oh, The Places You'll Go!","{'Childrens': 2345, 'Childrens-Picture Books':..."
8,11221285,4.946124,Brandon Sanderson,"The Way of Kings, Part 2 (The Stormlight Archi...","{'Fantasy': 641, 'Fiction': 46, 'Fantasy-Epic ..."
9,862041,4.942287,J.K. Rowling,"Harry Potter Boxset (Harry Potter, #1-7)","{'Fantasy': 3211, 'Young Adult': 1068, 'Fictio..."


---

# === Базовые подходы: коллаборативная фильтрация

# === Базовые подходы: контентные рекомендации

# === Базовые подходы: валидация

# === Двухстадийный подход: метрики

# === Двухстадийный подход: модель

# === Двухстадийный подход: построение признаков